##데이터 불러오기

In [ ]:
!pip install nlp
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip3 install kobert-transformers
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install -r requirements.txt

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 243 kB 73.3 MB/s 
     |████████████████████████████████| 46.9 MB 111 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595717 sha256=bad7ff502d989f1cb97474f4aa3b41d240206474b0dc22c4429cf7ffdef9326d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 3.3 MB 4.9 MB/s 
     |████████████████████████████████| 1.2 MB 56.3 MB/s 
     |████████████████████████████████| 3.3 MB 55.4 MB/s 
     |████████████████████████████████| 61 kB 675 kB/s 
     |████████████████████████████████| 596 kB 58.4 MB/s 
     |████████████

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim

from torch import nn
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
import numpy as np

#kobert
#from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, _ = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
df = pd.read_json('/content/drive/MyDrive/정비메모분석/data/desc_token_unique_labeled.json')
df

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%load_ext google.colab.data_table
df.groupby('label').count()

,desc_token
label,
AGM,25
AS,297
AS 진행,89
BNK,61
CSA,55
...,...
후방카메라 전원,40
후방카메라 정상,40
후패드,53


In [ ]:
print(len(df))
df = df.dropna()
print(len(df))

18743
18743


## 데이터셋 구성

In [ ]:
words = set()
for s in df.desc_token:
    words.update(s.split())

vocab = ['<UNK>', '<PAD>', '<BOS>', '<EOS>']
vocab.extend(words)
len(vocab)

2268

In [ ]:
word_to_num = dict()
num_to_word = dict()

for i, label in enumerate(vocab):
    word_to_num[label] = i
    num_to_word[i] = label

len(num_to_word)

2268

In [ ]:
MAX_LEN = max([len(x) for x in df.desc_token])
print(MAX_LEN)

10


18743

In [ ]:
pad = word_to_num['<PAD>']
bos = word_to_num['<BOS>']
eos = word_to_num['<EOS>']
s_info = np.array([0] * MAX_LEN)

def transform(s, max_len=MAX_LEN):
    s = [bos]+[word_to_num[x] for x in s.split()]+[eos]
    return (np.array(s + [pad]*(MAX_LEN-len(s))), np.array(len(s)), s_info)

x_data = [transform(s) for s in df.desc_token]
len(x_data)

18743

In [ ]:
label_to_num = dict()
num_to_label = dict()

for i, label in enumerate(df.label.unique()):
    label_to_num[label] = i
    num_to_label[i] = label

len(num_to_label)

236

In [ ]:
y_data = [label_to_num[x] for x in df.label]
len(y_data)

18743

In [ ]:
from sklearn.model_selection import train_test_split

data = [(x + (y, )) for x, y in zip(x_data, y_data)]
train_data, test_data = train_test_split(data, test_size=0.1, random_state=666)

print(len(train_data))
print(len(test_data))

16868
1875


In [ ]:
BATCH_SIZE = 64

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=5)

print(len(train_dataloader))
print(len(test_dataloader))

264
30


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## BERT

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=len(num_to_label),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.bert.init_weights() # 모델 초기화
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# Setting parameters
warmup_ratio = 0.1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
EPOCHS = 10

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * EPOCHS
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
def predict(model, dataloader):    
    model.eval()

    test_eval=[]
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            test_eval.append(np.argmax(logits))

    return test_eval

## 학습

In [ ]:
for e in range(EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/264 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.5607476234436035 train acc 0.015625
epoch 1 batch id 201 loss 2.4414589405059814 train acc 0.24012748756218905
epoch 1 train acc 0.3323403303872054


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1 test acc 0.7209429824561403


...

  0%|          | 0/264 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.28918299078941345 train acc 0.921875
epoch 10 batch id 201 loss 0.12286161631345749 train acc 0.9743470149253731
epoch 10 train acc 0.974681712962963


  0%|          | 0/30 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7faf47d80710>
Traceback (most recent call last):
AssertionError: can only test a child process
    assert self._parent_pid == os.getpid(), 'can only test a child process'
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7faf47d80710>
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/

epoch 10 test acc 0.9485745614035088


In [ ]:
from sklearn.metrics import f1_score

y_true = [x[3] for x in test_data]
y_pred = predict(model, test_dataloader)
f1_score(y_true, y_pred, average='weighted')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0.9495949235742448

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.79      0.86        19
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         7
           4       0.97      1.00      0.98        30
           5       1.00      1.00      1.00         2
           6       0.96      1.00      0.98        24
           8       1.00      0.91      0.95        11
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       0.97      1.00      0.99        37
          14       1.00      1.00      1.00         7
          15       1.00      1.00      1.00         5
          16       1.00      1.00      1.00        23
          17       0.75      1.00      0.86         3
          18       0.89    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

##테스트

In [ ]:
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)})

In [ ]:
def predict_sentence(sentence):
    preprocessed_sentence = sentence # 전처리 했다고 가정
    #preprocessed_sentence = preprocess_sentence(sentence)

    data = [(transform(preprocessed_sentence) + (0, ))]
    dataloader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE, num_workers=5)
    y_pred = predict(model, dataloader)
    return [num_to_label[x] for x in y_pred]

In [ ]:
predict_sentence('핸들 회전 작동 이상')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


['점검 필요']